In [1]:
import os
import pyspark.mllib
import numpy as np
import matplotlib.pyplot
import seaborn as sns
import pyspark
import urllib
import pandas as pd
import datetime

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as func
from pyspark.mllib.regression import LinearRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from IPython.display import display, HTML
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, VectorIndexer
import numpy as np
from numpy import array
%matplotlib inline
# get_ipython().magic(u'pylab inline')

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
### Initialize streaming context
conf = SparkConf()\
                .setMaster("local[2]")\
                .setAppName("MobileAnalyticsFeaturesExtraction")\
                .set("spark.executor.memory", "6g")\
                .set("spark.driver.memory", "6g")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName("spark play").getOrCreate()

In [4]:
phone_brand_model_csv = "data/phone_brand_device_model_key.csv"
gender_age_train_csv = "data/gender_age_train.csv"
gender_age_test_csv = "data/gender_age_test.csv"
events_csv = "data/events.csv"
app_events_csv = "data/app_events.csv"
app_labels_csv ="data/app_labels.csv"
label_categories_csv = "data/label_categories.csv"

train_CSV ="data/train"
test_CSV ="data/test"

In [5]:
app_events = spark.read.csv(app_events_csv, header=True, mode="DROPMALFORMED", inferSchema='true', encoding="utf-8").persist()

In [7]:
app_events = app_events.sample(False, 0.001, seed=0).persist()
print app_events.count()
app_events.show()

30
+--------+--------------------+------------+---------+
|event_id|              app_id|is_installed|is_active|
+--------+--------------------+------------+---------+
|   90431| 5729517255058371973|           1|        0|
|  103974|-5305696816021977482|           1|        0|
|  349994| 5927333115845830913|           1|        0|
|  430665|  596528045739680432|           1|        0|
|  432625|-6373901123647546992|           1|        0|
|  625541|-6538188762242952077|           1|        1|
|  628514| 5241322596077884489|           1|        0|
|  681830| 6989638496562408848|           1|        1|
| 1184963| 4348659952760821294|           1|        1|
| 1466930| 4828548453277771511|           1|        0|
| 1587761| 6666573788039351108|           1|        0|
| 1638126| 2555734998750410231|           1|        1|
| 1643442| -653184324994839917|           1|        0|
| 1709074| 8948670408023620661|           1|        1|
| 1734006|  821634886970639617|           1|        0|
| 18034

In [9]:
app_events.repartition(1).write.option("header", True).csv("demo/data/app_events")

In [10]:
events = spark.read.csv(events_csv, header=True, mode="DROPMALFORMED", inferSchema='true', encoding="utf-8").persist()
events = events.sample(False, 0.001, seed=0).persist()
print events.count()
events.show()

3257
+--------+--------------------+-------------------+---------+--------+
|event_id|           device_id|          timestamp|longitude|latitude|
+--------+--------------------+-------------------+---------+--------+
|     978| 2692075908067099101|2016-05-07 00:26:48|      0.0|     0.0|
|    1105| 4278305343584179554|2016-05-01 01:50:42|   126.79|   42.38|
|    3499|-7655873196685547711|2016-05-07 05:50:01|      0.0|     0.0|
|    4302|-8783851385933673163|2016-05-06 06:00:30|   118.81|   32.11|
|    4534|  125284895937135417|2016-05-07 06:41:48|   112.96|   30.45|
|    5232| -914879838163722787|2016-05-03 07:22:01|   117.14|    34.8|
|    7194| 9219164468944552013|2016-05-03 09:18:22|   120.17|   30.27|
|    7247| 5132594033699421887|2016-05-03 09:29:12|      0.0|     0.0|
|    7624| 4549750073184785510|2016-05-05 09:13:38|   121.61|   31.14|
|    9568| 1835470893908525178|2016-05-06 10:41:20|    113.7|   23.06|
|    9895| 8680066598920738422|2016-05-07 09:59:40|      0.0|     0.0|
|

In [11]:
events.repartition(1).write.option("header", True).csv("demo/data/events")

In [12]:
gender_age_train = spark.read.csv(gender_age_train_csv, header=True, mode="DROPMALFORMED", inferSchema='true', encoding="utf-8").persist()
gender_age_train = gender_age_train.sample(False, 0.001, seed=0).persist()
print events.count()
gender_age_train.show()

3257
+--------------------+------+---+------+
|           device_id|gender|age| group|
+--------------------+------+---+------+
|   11491559839923177|     M| 29|M29-31|
| 2450350084517153299|     F| 26|F24-26|
| 6707328039178412823|     M| 34|M32-38|
| 8686488919160645539|     M| 37|M32-38|
| 5487968851166251891|     F| 36|F33-42|
| 4236671290028993261|     M| 25|M23-26|
|-2920089512121882742|     F| 21|  F23-|
|-3927905525213459148|     M| 41|  M39+|
|   74758162773232977|     M| 20|  M22-|
|-1749090368336092788|     F| 24|F24-26|
| 3751556080923780612|     M| 21|  M22-|
|  395937639874086192|     F| 50|  F43+|
|-7936738974721311568|     F| 21|  F23-|
| 8652547644062704081|     M| 48|  M39+|
|-5365668398628094418|     F| 34|F33-42|
|-9058976870986022312|     M| 41|  M39+|
| 4975770075842412927|     M| 51|  M39+|
|-4198585308778054407|     F| 28|F27-28|
|-6843338455486182856|     M| 20|  M22-|
| 6633527966857717003|     F| 47|  F43+|
+--------------------+------+---+------+
only showin

In [13]:
gender_age_train.repartition(1).write.option("header", True).csv("demo/data/gender_age_train")